In [1]:
wdir="AM_videogames/"

import pandas as pd
import numpy as np
import time
import spacy
import pickle
# sp_trf_vector = spacy.load('en_core_web_trf', disable[])
# spacy.require_gpu()
sp_trf = spacy.load('en_core_web_trf')

# https://applied-language-technology.readthedocs.io/en/latest/notebooks/part_iii/04_embeddings_continued.html

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Mar_21_19:24:09_Pacific_Daylight_Time_2021
Cuda compilation tools, release 11.3, V11.3.58
Build cuda_11.3.r11.3/compiler.29745058_0


In [3]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.0.6                         
Location         C:\Users\basil\miniconda3\envs\basic\lib\site-packages\spacy
Platform         Windows-10-10.0.19041-SP0     
Python version   3.9.4                         
Pipelines        en_core_web_md (3.0.0), en_core_web_trf (3.0.0)



In [2]:
# %%timeit -n 1 -r 1 # number of loop


# POSLemmaChoice="SP" # WN TB SP
wdir="AM_videogames/"
chosen_game="diablo III"


basic_df = pd.read_pickle(wdir+"1_df.pkl") # 2 is sentence split, with stopwords 
basic_df=basic_df.loc[basic_df["game_title"]==chosen_game]
basic_df=basic_df.reset_index(drop=True)
# basic_df.head()


df_len=len(basic_df.index)
print(df_len)
percent=int(df_len/100)
doc_list=[]
for index, row in basic_df.iterrows():
    
    if (index%percent)==1:
        print(int(index/df_len*100) , " % processed", end="\r", flush=True)

    doc = sp_trf(row["review_text"])
    doc_list.append(doc)
# print(len(doc_data)) 

''' Serialization '''
# Serialize vocab (actually the whole NLP ojbect)
pickle.dump(doc_list, open(wdir+chosen_game+"_SP_trf.pickle", "wb"))

''' Deserialization '''
loaded_doc_list = pickle.load(open(wdir+chosen_game+"_SP_trf.pickle", "rb"))
print(len(loaded_doc_list))

593
593 % processed


In [4]:
# POSLemmaChoice="SP" # WN TB SP
wdir="AM_videogames/"
chosen_game="diablo III"
chosen_game_list=[]
used_games=["Company of Heroes 2",      ## games with most reviews ( around 100 each ) in the dataset     
"Diablo III",                     
"Sid Meier's Civilization VI",   
"SUPERHOT",                       
"XCOM 2"]  

basic_df = pd.read_pickle(wdir+"1_df.pkl") # 2 is sentence split, with stopwords 
basic_df=basic_df.loc[basic_df["game_title"]==chosen_game]
basic_df=basic_df.reset_index(drop=True)
basic_df.head(10)
texts=[]
for index,row in basic_df.iterrows():
    texts.append(row["review_text"])

In [5]:
%%timeit -n 1 -r 1 # number of loop
i=0
doc_list=[]
texts_number=len(texts)
for doc in sp_trf.pipe(texts, batch_size=64):
    i+=1
    print(int(i/texts_number*100) , " % processed", end="\r", flush=True)
    doc_list.append(doc)

''' Serialization '''
# Serialize vocab (actually the whole NLP ojbect)
pickle.dump(doc_list, open(wdir+chosen_game+"_SP_trf.pickle", "wb"))

''' Deserialization '''
loaded_doc_list = pickle.load(open(wdir+chosen_game+"_SP_trf.pickle", "rb"))
print(len(loaded_doc_list))

593  % processed % processed  % processed
2min 29s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [6]:
#https://applied-language-technology.readthedocs.io/en/latest/notebooks/part_iii/04_embeddings_continued.html
# great explanations of spacy transformers objects 

test= sp_trf("The cat is black.")

print(len(test)) # 5 tokens
    
print(test._.trf_data.tensors[0].shape) # outputs for 7 tokens bbecause trf works with sub-words
print(test._.trf_data.tensors[1].shape) # whole doc output
# Check the first ten dimensions of the tensor


# [0] to enter the batch ( only one batch in this case), then print first 10 dimensions of each token
# print(test._.trf_data.tensors[0][0][:10] )
token_index=0
print(test[token_index], test._.trf_data.align[token_index].data)

5
(1, 7, 768)
(1, 768)
The [[1]]


In [7]:
sp_trf.pipeline

[('transformer',
  <spacy_transformers.pipeline_component.Transformer at 0x1a3304baea0>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x1a33050d400>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x1a3304eef40>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x1a330547400>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x1a330534880>),
 ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x1a330554c80>)]

In [9]:
''' Deserialization '''
loaded_doc_list = pickle.load(open(wdir+chosen_game+"_SP_trf.pickle", "rb"))
print(len(loaded_doc_list))


593


In [15]:
loaded_doc_list[2]._.trf_data.tensors[0].shape

(8, 140, 768)